### Naver Movie
- robots.txt 설정으로 크롤링이 안됩니다.



#### 1. 프로젝트 생성

In [1]:
!scrapy startproject naver_movie

New Scrapy project 'naver_movie', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/scrapy/naver_movie

You can start your first spider with:
    cd naver_movie
    scrapy genspider example example.com


#### 2. Items 설정
- https://movie.naver.com/movie/running/current.nhn
- 제목, 관객수, 평점

In [3]:
! cat naver_movie/naver_movie/items.py

# -*- coding: utf-8 -*-

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class NaverMovieItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass


In [4]:
%%writefile naver_movie/naver_movie/items.py
import scrapy

class NaverMovieItem(scrapy.Item):
    # define the fields for your item here like:
    title = scrapy.Field()
    count = scrapy.Field()
    star = scrapy.Field()

Overwriting naver_movie/naver_movie/items.py


#### 3. xpath 확인

In [7]:
import requests
import scrapy
from scrapy.http import TextResponse


In [9]:
# 링크
req = requests.get("https://movie.naver.com/movie/running/current.nhn")
response = TextResponse(req.url, body=req.text, encoding='utf-8')

In [11]:
links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()
len(links), links[0]


(114, '/movie/bi/mi/basic.nhn?code=136873')

In [13]:
link = response.urljoin(links[0])
link

'https://movie.naver.com/movie/bi/mi/basic.nhn?code=136873'

In [14]:
# 상세 데이터 수집
req = requests.get(link)
response = TextResponse(req.url, body=req.text, encoding='utf-8')

In [17]:
title = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0]
count = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()').extract()[0]
star = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
star = "".join(star)
title, count, star

('겨울왕국 2', '607,850명', '9.23')

#### 4. spider 작성


In [25]:
#내가작성
%%writefile naver_movie/naver_movie/spiders/spider.py
import scrapy
from naver_movie.items import NaverMovieItem

class MovieSpider(scrapy.Spider):
    name = "MovieSpider"
    allow_domain = ["https://movie.naver.com/"]
    start_url = ["https://movie.naver.com/movie/running/current.nhn"]
    
    def parse(self,response):
        links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()
        link = [response.urljoin(link) for link in links]
        for link in links:
            yield scrapy.Request(link, callback=self.page_content)
            
    def page_content(self, response):
        item = NaverMovieItem()
        item['title'] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0]
        try:
            item['count'] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()').extract()[0]
        except:
            item['count'] = "0명"
        item['star'] = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
        item['star'] = "".join(star)
        
        yield item

Overwriting naver_movie/naver_movie/spiders/spider.py


In [31]:
#수업
%%writefile naver_movie/naver_movie/spiders/spider.py
import scrapy
from naver_movie.items import NaverMovieItem
class MovieSpider(scrapy.Spider):
    name = "NaverMovie"
    allow_domain = ["https://movie.naver.com"]
    start_urls = ["https://movie.naver.com/movie/running/current.nhn"]
    def parse(self, response):
        links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href')[:10].extract()
        for link in links:
            link = response.urljoin(link)
            yield scrapy.Request(link, callback=self.parse_page_contents)
    def parse_page_contents(self, response):
        item = NaverMovieItem()
        item["title"] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0]
        try:
            item["count"] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()').extract()[0]
        except:
            item["count"] = "0명"
        star = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
        item["star"] = "".join(star)
        yield item

Overwriting naver_movie/naver_movie/spiders/spider.py


#### 5.Scrapy 실행

In [34]:
%%writefile run.sh
cd naver_movie
scrapy crawl "NaverMovie" -o naver_movie.csv

Overwriting run.sh


In [35]:
!./run.sh

2019-11-22 08:12:01 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: naver_movie)
2019-11-22 08:12:01 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.6.9 (default, Oct 24 2019, 05:23:48) - [GCC 7.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.15.0-1054-aws-x86_64-with-debian-buster-sid
2019-11-22 08:12:01 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'naver_movie', 'FEED_FORMAT': 'csv', 'FEED_URI': 'naver_movie.csv', 'NEWSPIDER_MODULE': 'naver_movie.spiders', 'SPIDER_MODULES': ['naver_movie.spiders']}
2019-11-22 08:12:01 [scrapy.extensions.telnet] INFO: Telnet Password: bec6c05a06542985
2019-11-22 08:12:01 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.log

#### 6. settings.py 파일 변경
- Forbidden by robots.txt

In [36]:
!head -n 25 naver_movie/naver_movie/settings.py | tail -n5

# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32


In [37]:
!sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' naver_movie/naver_movie/settings.py

In [38]:
!./run.sh

2019-11-22 08:12:09 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: naver_movie)
2019-11-22 08:12:09 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.6.9 (default, Oct 24 2019, 05:23:48) - [GCC 7.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.15.0-1054-aws-x86_64-with-debian-buster-sid
2019-11-22 08:12:09 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'naver_movie', 'FEED_FORMAT': 'csv', 'FEED_URI': 'naver_movie.csv', 'NEWSPIDER_MODULE': 'naver_movie.spiders', 'SPIDER_MODULES': ['naver_movie.spiders']}
2019-11-22 08:12:09 [scrapy.extensions.telnet] INFO: Telnet Password: f0c94dd15dbfa697
2019-11-22 08:12:09 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.log